# Plot N cross-section with matplotlib

In [1]:
%matplotlib widget

In [50]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import numpy as np

In [4]:
ncfilename = '/Users/julesblom/ThesisResults/MultiTest/trim-36km_200m_W60ChannelRun01_compressed_realclean.nc'
trim = xr.open_dataset(ncfilename) # Lazyload DataSet with xarray, unload from memory with .close()

In [5]:
# only if the NetCDF has already been processed
if "depth_center" in trim:
    depth_center = trim['depth_center']
else:
    print("processNetCDF first")

In [6]:
selected_time = 1
selected_M = int(trim.M.size/2)

density_m_at_time = trim.RHO.isel(time=selected_time, M=selected_M) 
z_section_center = depth_center.isel(time=selected_time, M=selected_M) # density_m_31.depth_center.T #trim.depth_center.isel(time=this_time, M=this_M).T # why do i need to transpose this?

z_section_intf = trim.depth.isel(time=selected_time, M=selected_M) # why do i need to transpose this?

n_section = z_section_intf.YZ

_, mesh_N = np.meshgrid(trim.SIG_INTF.values, n_section)

### Sanity check of dimensions

In [7]:
# is there a way to quickly print a formatted table in jupyter without pandas?
print('quad_NSigma\t', mesh_N.shape)
print('n_section\t', n_section.shape)
print('z_section_center', z_section_center.shape)
print('z_section_intf\t', z_section_intf.shape)
print('density_m_at_time', density_m_at_time.T.shape)

quad_NSigma	 (182, 81)
n_section	 (182,)
z_section_center (182, 80)
z_section_intf	 (182, 81)
density_m_at_time (182, 80)


In [8]:
sigma_percentages = np.diff(trim.SIG_LYR.values)*-100

In [9]:
z_section_center.isel(N=-1).diff('SIG_LYR').to_dataframe()

,XZ,YZ,time,depth_center,depth_center
SIG_LYR,,,,,
-0.10180,13100.0,36200.0,2020-02-01 01:42:00,-71.250549,-46.123882
-0.15930,13100.0,36200.0,2020-02-01 01:42:00,-111.495209,-40.244659
-0.21180,13100.0,36200.0,2020-02-01 01:42:00,-148.240326,-36.745117
-0.25930,13100.0,36200.0,2020-02-01 01:42:00,-181.485916,-33.245590
-0.30180,13100.0,36200.0,2020-02-01 01:42:00,-211.231995,-29.746078
...,...,...,...,...,...
-0.99840,13100.0,36200.0,2020-02-01 01:42:00,-698.787292,-0.314941
-0.99880,13100.0,36200.0,2020-02-01 01:42:00,-699.067261,-0.279968
-0.99920,13100.0,36200.0,2020-02-01 01:42:00,-699.347229,-0.279968


In [10]:
# newDF = pd.DataFrame(section)

# display(s)


In [11]:
legend_hack_intf = z_section_intf.isel(SIG_INTF=-1)
legend_hack_center = z_section_center.isel(SIG_LYR=-1)

In [12]:
3620.0/700

5.171428571428572

In [13]:
plt.close("all")

## Plot Sigma interfaces and center with length

In [52]:
# zscale = 20
fig, ax = plt.subplots(nrows=1, figsize=(10,4.5)) # scale to actual depth lenght: (20 ,4) for  factor 10
plt.title("Channel slope 1.5° - Basin slope 0.2°")
plt.suptitle('Sigma grid vertical section - Length (n)')

ax.plot(n_section, z_section_intf, c='k', linewidth=0.55, marker='+', markersize=0.1)
ax.plot(n_section, z_section_center, c='steelblue', linewidth=0.15, marker='+', markersize=0.1)

ax.xaxis.set_minor_locator(MultipleLocator(1000))
plt.xlim(0., 36200)

# hack to get only names in legend for lines
interfaces, = ax.plot(n_section, legend_hack_intf, c='k', linewidth=0.8, marker='+', markersize=0.)
centers, = ax.plot(n_section, legend_hack_center, c='steelblue', linewidth=0.8, marker='+', markersize=0.1)

# mark one line just because
# ax.plot(n_section, z_section_intf[:,42], c='hotpink', marker='o', markersize=1.5,)


# inset axes
axins = ax.inset_axes([0.5, 0.5, 0.47, 0.47])
axins.plot(n_section, z_section_intf, c='k', linewidth=0.55, marker='+', markersize=0.1)

# axins.imshow(Z2, extent=extent, interpolation="nearest",origin="lower")
# sub region of the original image
x1, x2, y1, y2 = 13000, 17000, -640, -600
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')

ax.indicate_inset_zoom(axins, edgecolor='tomato', alpha=0.5)

# ax.scatter(n_section, depth_m_31[:,10], s=1.5)
ax.set_xlabel('Length [m]', fontsize=13.5)
ax.set_ylabel('Depth [m]', fontsize=13.5)
ax.legend((interfaces, centers), ('$\sigma$ Interfaces', '$\sigma$ Centers'), fontsize="x-large", loc="lower left", markerscale=50)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
help(ax.indicate_inset_zoom)
# plt.close('all')

Help on method indicate_inset_zoom in module matplotlib.axes._axes:

indicate_inset_zoom(inset_ax, **kwargs) method of matplotlib.axes._subplots.AxesSubplot instance
    Add an inset indicator rectangle to the axes based on the axis
    limits for an *inset_ax* and draw connectors between *inset_ax*
    and the rectangle.
    
    Warnings
    --------
    This method is experimental as of 3.0, and the API may change.
    
    Parameters
    ----------
    inset_ax : `.Axes`
        Inset axes to draw connecting lines to.  Two lines are
        drawn connecting the indicator box to the inset axes on corners
        chosen so as to not overlap with the indicator box.
    
    **kwargs
        Other keyword arguments are passed on to `.Axes.indicate_inset`
    
    Returns
    -------
    rectangle_patch : `.Patches.Rectangle`
         Rectangle artist.
    
    connector_lines : 4-tuple of `.Patches.ConnectionPatch`
        Each of four connector lines coming from the rectangle drawn on

## Plot Sigma interfaces and center with grid index

In [20]:
fig_N, ax_N = plt.subplots(nrows=1, figsize=(9,6))
ax_N.set_title('Sigma layers vertical section - N number')

ax_N.plot(z_section_intf, c='k', linewidth=0.25, marker='+', markersize=0.1)
ax_N.plot(z_section_center, c='g', linewidth=0.05, marker='+', markersize=0.1)
ax_N.set_xlabel('N')
ax_N.set_ylabel('Depth [m]')
fig_N.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot density N cross section

In [ ]:
print(mesh_N[:,:].shape)
print(z_section_intf[:,:].shape)
print(density_m_31.T.shape)

In [ ]:
fig_vert, ax_vert1 = plt.subplots(nrows=1, figsize=(9,6))

ax_vert1.set_title('Vertical section density')

# or z_section.YZ both work
mesh = ax_vert1.pcolormesh(mesh_N, z_section_intf, density_m_31.T,
                          vmin=1025, vmax=1025.5) #, edgecolors='w', linewidths=0.1)
ax_vert1.plot(n_section, z_section_center, c='lightsteelblue', linewidth=0.10) #, marker='+', markersize=0.1)
ax_vert1.plot(n_section, z_section_intf[:,:], c='bisque', linewidth=0.08)
# ax_vert1.plot(n_section, z_section_intf[:,45], c='hotpink', marker='o', markersize=0.75,)
cbar = fig_vert.colorbar(mesh, ax=ax_vert1)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Density [kg/m3]',rotation=90)
ax_vert1.set_xlabel('N [m]')
ax_vert1.set_ylabel('Depth [m]')